In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import copy
import tensorflow as tf
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
dataset_cols = ["Date", "Bike_count" ,"Hour" ,"Temp" ,"Humidity" ,"Windspeed" ,"Visibility" ,"Dew_pt_temp" ,"radiation" ,"Rainfall" ,"Snowfall" ,"Seasons" ,"Holiday" ,"Functional"]
df = pd.read_csv("SeoulBikeData.csv", encoding='latin-1')
df.columns = dataset_cols
df = df.drop(["Date", "Holiday", "Seasons"], axis=1)

In [ ]:
df.head()

In [ ]:
df["Functional"] = (df["Functional"] == "Yes").astype(int)
df = df[df["Hour"] == 12]
df = df.drop(["Hour"], axis=1)
df.head()

In [ ]:
for label in df.columns[1:]:
  plt.scatter(df[label], df["Bike_count"])
  plt.title(label)
  plt.ylabel("Noon Bike Count")
  plt.xlabel(label)
  plt.show()

In [ ]:
df = df.drop(["Windspeed", "Visibility", "Functional"], axis=1)
df.head()

#Train/Validate/Test dataset

In [ ]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [ ]:
def get_xy(dataframe, y_labels, x_labels=None):
  dataframe = copy.deepcopy(dataframe)#copying the values from the dataframe.
  if x_labels is None:
    X = dataframe[[c for c in dataframe.columns if c!=y_labels]].values
  else:
    if len(x_labels) == 1:
      X = dataframe[x_labels[0]].values.reshape(-1, 1)
    else:
      X = dataframe[x_labels].values

    y = dataframe[y_labels].values.reshape(-1,1)
    data = np.hstack((X,y))

    return data, X, y

In [ ]:
_, X_train, y_train = get_xy(train, "Bike_count", x_labels=["Temp"])
_, X_val, y_val = get_xy(val, "Bike_count", x_labels=["Temp"])
_, X_test, y_test = get_xy(test, "Bike_count", x_labels=["Temp"])

In [ ]:
temp_reg = LinearRegression() #initialising the linear regression model.
temp_reg.fit(X_train, y_train)

In [ ]:
#print(temp_reg.coef_, temp_reg.intercept_)
temp_reg.score(X_train, y_train)

In [ ]:
plt.scatter(X_train, y_train, label="data", color="blue")
x = tf.linspace(-20, 40, 100)
plt.plot(x, temp_reg.predict(np.array(x).reshape(-1, 1)), label="fit", color='red', linewidth=3)
plt.legend()
plt.title("Bikes vs Temp")
plt.ylabel("No. of bikes")
plt.xlabel("Temp")
plt.show()

#Multiple Linear Regression

In [ ]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
_, X_train_all, y_train_all = get_xy(train, "Bike_count", x_labels=df.columns[1:])
_, X_val_all, y_val_all = get_xy(val, "Bike_count", x_labels=df.columns[1:])
_, X_test_all, y_test_all = get_xy(test, "Bike_count", x_labels=df.columns[1:])

In [ ]:
all_reg = LinearRegression()
all_reg.fit(X_train_all, y_train_all)

In [ ]:
all_reg.score(X_test_all, y_test_all)

#Regression using Neural Networks

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('MSE')
  plt.grid(True)
  plt.show()

In [ ]:
temp_normalizer = tf.keras.layers.Normalization(input_shape=(1,), axis=None)
temp_normalizer.adapt(X_train.reshape(-1))

In [ ]:
temp_nn_model = tf.keras.Sequential([
    temp_normalizer,
    tf.keras.layers.Dense(1)
])

In [ ]:
temp_nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mean_squared_error')

In [ ]:
history = temp_nn_model.fit(
    X_train.reshape(-1), y_train,
    verbose=0,
    epochs=1000,
    validation_data=(X_val, y_val)
)

In [ ]:
plot_loss(history)

In [ ]:
plt.scatter(X_train, y_train, label="data", color="blue")
x = tf.linspace(-20, 40, 100)
plt.plot(x, temp_nn_model.predict(np.array(x).reshape(-1, 1)), label="fit", color='red', linewidth=3)
plt.legend()
plt.title("Bikes vs Temp")
plt.ylabel("No. of bikes")
plt.xlabel("Temp")
plt.show()

In [ ]:
temp_normalizer = tf.keras.layers.Normalization(input_shape=(1,), axis=None)
temp_normalizer.adapt(X_train.reshape(-1))

nn_model = tf.keras.Sequential([
    temp_normalizer,
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
history = nn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    verbose=0, epochs=100
)

In [ ]:
plot_loss(history)

In [ ]:
plt.scatter(X_train, y_train, label="data", color="blue")
x = tf.linspace(-20, 40, 100)
plt.plot(x, nn_model.predict(np.array(x).reshape(-1, 1)), label="fit", color='red', linewidth=3)
plt.legend()
plt.title("Bikes vs Temp")
plt.ylabel("No. of bikes")
plt.xlabel("Temp")
plt.show()

In [ ]:
all_normalizer = tf.keras.layers.Normalization(input_shape=(6,), axis=-1)
all_normalizer.adapt(X_train_all)

In [ ]:
from tensorflow.keras.layers import Input

input_layer = Input(shape=(6,))
normalized_input = all_normalizer(input_layer)
dense_layers = tf.keras.layers.Dense(32, activation='relu')(normalized_input)
dense_layers = tf.keras.layers.Dense(32, activation='relu')(dense_layers)
output_layer = tf.keras.layers.Dense(1)(dense_layers)

nn_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
history = nn_model.fit(
    X_train_all, y_train_all,
    validation_data=(X_val_all, y_val_all),
    verbose=0, epochs=100
)

In [ ]:
plot_loss(history)

In [ ]:
#calculating MSE for both linear reg and nn.
y_pred_lr = all_reg.predict(X_test_all)
y_pred_nn = nn_model.predict(X_test_all)

In [ ]:
def MSE(y_pred, y_real):
  return (np.square(y_pred-y_real)).mean()

In [ ]:
MSE(y_pred_lr, y_test_all)

In [ ]:
ax = plt.axes(aspect="equal")
plt.scatter(y_test_all, y_pred_lr, label="Lin Reg preds")
plt.scatter(y_test_all, y_pred_nn, label="NN preds")
plt.xlabel("true values")
plt.ylabel("predictions")
lims = [0, 2500]
plt.xlim(lims)
plt.ylim(lims)
plt.legend()
_= plt.plot(lims, lims, c="red")